In [1]:
from z3 import *
import itertools
from tqdm import tqdm

## Sorts

In [2]:
# create uninterpreted sort Node and Epoch
Node = DeclareSort('Node')
Epoch = DeclareSort('Epoch')

In [3]:
# Functions to extract terms and instantiate quantifiers
def extract_terms(expr, bound=[]):
    termdict = {Node: set(), Epoch: set()}
    if expr.sort() in {Node, Epoch}:
        termdict[expr.sort()] = termdict[expr.sort()] | {expr}    
    if type(expr) == QuantifierRef:
            raise ValueError('Unsupported')
    termdict_children = [extract_terms(child) for child in expr.children()]
    for child_termdict in termdict_children:
        termdict[Node] = termdict[Node] | child_termdict[Node]
        termdict[Epoch] = termdict[Epoch] | child_termdict[Epoch]
    return termdict

def instantiate_old(expr, termdict):
#     if type(expr) != QuantifierRef or not expr.is_forall():
#         raise ValueError('Unsupported')
    quantified_variables, body = expr #expr.body()
    if not isinstance(body,ExprRef):
        raise ValueError('Unsupported')
    #quantified_variables = [Const(expr.var_name(i), expr.var_sort(i)) for i in range(expr.num_vars())]
    instantiations = itertools.product(*[termdict[var.sort()] for var in quantified_variables])
    print(f'instantiating {[len(termdict[var.sort()]) for var in quantified_variables]}')
#     print(f'quant_vars: {quantified_variables}')
#     print(f'instantiations: {list(instantiations)}')
#     print(f'body: {body}')
#     print(f'instance: {substitute(body, list(zip(quantified_variables, list(instantiations[0]))))}')
#     print(f'zip: {list(zip(quantified_variables, list(instantiations[0])))}')
    return And([substitute(body, list(zip(quantified_variables, list(inst)))) for inst in tqdm(instantiations)])

def instantiate(expr, termdict):
    # returns generator of instantiations
    quantified_variables, body = expr
    if not isinstance(body,ExprRef):
        raise ValueError('Unsupported')
    
    instantiations = itertools.product(*[termdict[var.sort()] for var in quantified_variables])
    print(f'instantiating {[len(termdict[var.sort()]) for var in quantified_variables]}')

    mapper = lambda inst: substitute(body, list(zip(quantified_variables, list(inst))))
    return map(mapper, instantiations) # for inst in tqdm(instantiations)]

## State and Model

In [4]:
class DistLockState():
    def __init__(self, name):
        self.name = name

        # relations
        self.ep = Function(f'{name}.ep', Node, Epoch)
        self.held = Function(f'{name}.held', Node, BoolSort())
        self.transfer = Function(f'{name}.transfer', Epoch, Node, BoolSort())
        self.locked = Function(f'{name}.locked', Epoch, Node, BoolSort())

DistLockState('test_pre') # only for testing

In [5]:
class DistLockModel():
    def __init__(self):
        # constants
        self.le = Function(f'le', Epoch, Epoch, BoolSort())
        self.zero = Const(f'zero', Epoch)
        self.one = Const(f'one', Epoch)
        self.first = Const(f'first', Node)

        self.states = {}
    
    def get_state(self, name):
        if name not in self.states:
            self.states[name] = DistLockState(name)
        return self.states[name]
    
    def get_axioms(self):
        e1, e2, e3 = Consts('e1 e2 e3', Epoch)
        
        Axioms = ([e1, e2, e3], #ForAll([e1, e2, e3],
            And(
                # reflexivity
                self.le(e1, e1),
                # transitivity
                Implies(And(self.le(e1, e2), self.le(e2, e3)), self.le(e1, e3)),
                # antisymmetry
                Implies(And(self.le(e1, e2), self.le(e2, e1)), e1 == e2),
                # totality
                Or(self.le(e1, e2), self.le(e2, e1)),

                # zero
                self.le(self.zero, e1),
                self.one != self.zero,
            ),
        )

        return Axioms
    
    def get_init_state_cond(self):
        S = self.get_state('init')

        n = Const('n', Node)
        e = Const('e', Epoch)

        cond = ([n, e], #ForAll([n, e],
            And(
                S.held(n) == (n == self.first),
                Implies(n != self.first, S.ep(n) == self.zero),
                S.ep(self.first) == self.one,
                S.transfer(e, n) == False,
                S.locked(e, n) == False,
            )
        )

        return cond
    
    def get_interp(self, model: ModelRef):
        # create a dict of all the functions
        interp = {}
        for f in model.decls():
            interp[f.name()] = model.get_interp(f)
        
        return interp

M = DistLockModel()
# only for testing
axiom = M.get_axioms() 
terms = extract_terms(axiom[1])
print(terms)
list(instantiate(axiom, terms))
#axiom

{Node: set(), Epoch: {e2, one, e3, e1, zero}}
instantiating [5, 5, 5]


[And(le(e2, e2),
     Implies(And(le(e2, e2), le(e2, e2)), le(e2, e2)),
     Implies(And(le(e2, e2), le(e2, e2)), e2 == e2),
     Or(le(e2, e2), le(e2, e2)),
     le(zero, e2),
     one != zero),
 And(le(e2, e2),
     Implies(And(le(e2, e2), le(e2, one)), le(e2, one)),
     Implies(And(le(e2, e2), le(e2, e2)), e2 == e2),
     Or(le(e2, e2), le(e2, e2)),
     le(zero, e2),
     one != zero),
 And(le(e2, e2),
     Implies(And(le(e2, e2), le(e2, e3)), le(e2, e3)),
     Implies(And(le(e2, e2), le(e2, e2)), e2 == e2),
     Or(le(e2, e2), le(e2, e2)),
     le(zero, e2),
     one != zero),
 And(le(e2, e2),
     Implies(And(le(e2, e2), le(e2, e1)), le(e2, e1)),
     Implies(And(le(e2, e2), le(e2, e2)), e2 == e2),
     Or(le(e2, e2), le(e2, e2)),
     le(zero, e2),
     one != zero),
 And(le(e2, e2),
     Implies(And(le(e2, e2), le(e2, zero)), le(e2, zero)),
     Implies(And(le(e2, e2), le(e2, e2)), e2 == e2),
     Or(le(e2, e2), le(e2, e2)),
     le(zero, e2),
     one != zero),
 And(le(e2, e2

## Invariants

In [6]:
# Inv: le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))
# in other words: e1 < e2 ==> e1 <= ep(N1) or e2 > ep(N1)

def get_inv1(M: DistLockModel, S: DistLockState):
    e1, e2 = Consts('e1 e2', Epoch)
    n1 = Const('n1', Node)

    Inv = ([e1, e2, n1], #ForAll([e1, e2, n1],
        Implies(
            And(
                M.le(e1, e2),
                e1 != e2
            ),
            Or(
                M.le(e1, S.ep(n1)),
                Not(
                    M.le(e2, S.ep(n1))
                )
            )
        )
    )
    return Inv

M = DistLockModel()
S = M.get_state('pre')
# only for testing
inv1 = get_inv1(M, S)
inv1

([e1, e2, n1],
 Implies(And(le(e1, e2), e1 != e2),
         Or(le(e1, pre.ep(n1)), Not(le(e2, pre.ep(n1))))))

### Invariant parser

In [7]:
invariants = """
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E2,ep(N1))
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~locked(E2,N1)
le(E1, E2) & E1 ~= E2 -> locked(E1,N1) | ~transfer(E1,N1) | ~locked(E2,N1)
le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> ~le(E2,ep(N1)) | ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~locked(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~locked(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~held(N2) | ~transfer(E2,N1)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~locked(E2,N2) | ~le(ep(N2),ep(N1))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~transfer(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~transfer(E2,N2)
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E2,N1) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> le(E1,ep(N1)) | ~le(ep(N2),ep(N1)) | ~le(E2,ep(N2))
le(E1, E2) & E1 ~= E2 & N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~locked(E2,N2)
locked(E1,N1) | ~transfer(E1,N1) | ~le(E1,ep(N1))
locked(E1,N1) | ~held(N1) | ~transfer(E1,N1)
le(E1,ep(N1)) | ~held(N1) | ~transfer(E1,N1)
transfer(E1,N1) | ~locked(E1,N1)
le(E1,ep(N1)) | ~locked(E1,N1)
N1 ~= N2 -> ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1)) | ~first=N1
N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~locked(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | le(ep(N1),ep(N2)) | ~le(E1,ep(N2))
N1 ~= N2 -> le(ep(N1),ep(N2)) | ~held(N2)
N1 ~= N2 -> ~held(N1) | ~le(ep(N1),ep(N2))
N1 ~= N2 -> locked(E1,N1) | ~held(N2) | ~transfer(E1,N1)
N1 ~= N2 -> le(ep(N1),ep(N2)) | le(ep(N2),ep(N1))
N1 ~= N2 -> le(E1,ep(N1)) | ~locked(E1,N2) | ~le(ep(N2),ep(N1))
N1 ~= N2 -> ~held(N1) | ~held(N2)
N1 ~= N2 -> locked(E1,N1) | ~transfer(E1,N1) | ~le(E1,ep(N2))
N1 ~= N2 -> ~locked(E1,N1) | ~locked(E1,N2)
N1 ~= N2 -> ~first=N1 | ~first=N2
N1 ~= N2 -> ~transfer(E1,N1) | ~transfer(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~transfer(E1,N2)
N1 ~= N2 -> ~transfer(E1,N1) | ~locked(E1,N2)
N1 ~= N2 -> ~locked(E1,N1) | ~le(ep(N1),ep(N2)) | ~le(ep(N2),ep(N1))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~locked(E1,N2)
N1 ~= N2 -> le(E1,ep(N1)) | ~le(ep(N2),ep(N1)) | ~le(E1,ep(N2))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N1) | ~le(E1,ep(N2))
N1 ~= N2 -> le(E1,ep(N1)) | ~held(N2) | ~transfer(E1,N1)
N1 ~= N2 -> le(E1,ep(N1)) | ~transfer(E1,N1) | ~le(E1,ep(N2))
""".replace("~=", "!=").strip().split("\n")

def parse(inv_str, vars):
    global invariant_index
    
    inv_str = inv_str.strip()

    if inv_str.count("->") > 1:
        raise Exception("Too many ->")

    if '->' in inv_str:
        left, right = inv_str.split('->')
        return f"Implies({parse(left, vars)},{parse(right, vars)})"

    if "&" in inv_str:
        parts = inv_str.split("&")
        return f"And([{','.join([parse(p, vars) for p in parts])}])"

    if "|" in inv_str:
        parts = inv_str.split("|")
        return f"Or([{','.join([parse(p, vars) for p in parts])}])"

    if "~" in inv_str:
        return f"Not({parse(inv_str[1:], vars)})"

    if "!=" in inv_str:
        left, right = inv_str.split("!=")
        return f"{parse(left, vars)} != {parse(right, vars)}"
    
    if "=" in inv_str:
        left, right = inv_str.split("=")
        return f"{parse(left, vars)} == {parse(right, vars)}"

    if inv_str.endswith(")"):
        first_brace_idx = inv_str.index("(")
        name = inv_str[:first_brace_idx]
        args = inv_str[first_brace_idx+1:-1].split(",")
        prefix = "M." if name == 'le' else "S."
        return f"{prefix}{name}({','.join([parse(a, vars) for a in args])})"

    if inv_str == "first":
        return "M.first"

    vars.add(f'_{str(invariant_index)}{inv_str}')
    return f'_{str(invariant_index)}{inv_str}'

invariant_index = 0
def get_inv_fn(fn_name, inv_str, only_code=False):
    global invariant_index

    vars = set()
    inv_fn_str = parse(inv_str, vars)

    nodes = [v for v in vars if "N" in v]
    epochs = [v for v in vars if "E" in v]
    invariant_index += 1

    code = []
    if len(nodes) == 1:
        code += [nodes[0] + " = Const('" + nodes[0] + "', Node)"]
    elif len(nodes) > 1:
        code += [", ".join(nodes) + " = Consts('" + " ".join(nodes) + "', Node)"]
    
    if len(epochs) == 1:
        code += [epochs[0] + " = Const('" + epochs[0] + "', Epoch)"]
    elif len(epochs) > 1:
        code += [", ".join(epochs) + " = Consts('" + " ".join(epochs) + "', Epoch)"]
    
    code += ["inv = ([" + ", ".join(nodes + epochs) + "], " + inv_fn_str + ")"]
    code += ["return inv"]

    code = f"def {fn_name}(M, S):\n\t" + "\n\t".join(code)
    if only_code:
        return code

    ldict = {}
    exec(code)
    exec(f"ldict['fn'] = {fn_name}")
    return ldict['fn']

all_invars = [get_inv_fn("inv_fn_" + str(i), inv) for i, inv in enumerate(invariants)]

In [8]:
M = DistLockModel()
S = M.get_state('pre')

print(invariants[0])
all_invars[0](M, S)

le(E1, E2) & E1 != E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))


([_0N1, _0E2, _0E1],
 Implies(And(le(_0E1, _0E2), _0E1 != _0E2),
         Or(le(_0E1, pre.ep(_0N1)),
            Not(le(_0E2, pre.ep(_0N1))))))

### Safety Condition

In [9]:
# Inv: safety property locked(E, N1) & locked(E, N2) -> N1 = N2

def get_safety_inv(M: DistLockModel, S: DistLockState):
    e1 = Const('e1', Epoch)
    n1, n2 = Consts('n1 n2', Node)

    Inv = ([e1, n1, n2], #ForAll([e1, n1, n2],
        Implies(
            And(
                S.locked(e1, n1),
                S.locked(e1, n2)
            ),
            n1 == n2
        )
    )

    return Inv

M = DistLockModel()
S = M.get_state('pre')
get_safety_inv(M, S) # only for testing

([e1, n1, n2],
 Implies(And(pre.locked(e1, n1), pre.locked(e1, n2)),
         n1 == n2))

## Actions

In [10]:
# Grant action

def get_grant_action(M: DistLockModel, S1: DistLockState, S2: DistLockState):
    e = Const('e', Epoch)
    n1, n2 = Consts('n1 n2', Node)

    AcceptAction = ([n1, n2, e], #ForAll([n1, n2, e],
        Implies(
            # precondition
            And(
                S1.held(n1),
                Not(M.le(e, S1.ep(n1)))
            ),
            # postcondition
            And(
                S2.transfer(e, n2),
                S2.held(n1) == False
            )
        )
    )

    return AcceptAction

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
get_grant_action(M, S1, S2) # only for testing

([n1, n2, e],
 Implies(And(pre.held(n1), Not(le(e, pre.ep(n1)))),
         And(post.transfer(e, n2), post.held(n1) == False)))

In [11]:
# Accept action

def get_accept_action(M: DistLockModel, S1: DistLockState, S2: DistLockState):
    e = Const('e', Epoch)
    n = Const('n', Node)

    AcceptAction = ([n, e], #ForAll([n, e],
        Implies(
            And(
                S1.transfer(e, n),
                Not(M.le(e, S1.ep(n)))
            ),
            And(
                S2.held(n),
                S2.ep(n) == e,
                S2.locked(e, n)
            )
        )
    )

    return AcceptAction

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
get_accept_action(M, S1, S2) # only for testing

([n, e],
 Implies(And(pre.transfer(e, n), Not(le(e, pre.ep(n)))),
         And(post.held(n),
             post.ep(n) == e,
             post.locked(e, n))))

## VCs

In [12]:
def get_init_vc(M: DistLockModel, inv_fn):
    vc = ( #Not(
        Implies(
            ForAll(*M.get_axioms()),
            Implies(
                ForAll(*M.get_init_state_cond()),
                (inv_fn(M, M.get_state('init'))), # ForAll(*inv_fn(M, M.get_state('init')))
            )
        )
    )
    return vc

M = DistLockModel()
inv_fn = lambda M,S: ForAll(*get_inv1(M, S))
vc = get_init_vc(M, inv_fn)
print(vc.sexpr()) # only for testing

(let ((a!1 (forall ((e1 Epoch) (e2 Epoch) (e3 Epoch))
             (and (le e1 e1)
                  (=> (and (le e1 e2) (le e2 e3)) (le e1 e3))
                  (=> (and (le e1 e2) (le e2 e1)) (= e1 e2))
                  (or (le e1 e2) (le e2 e1))
                  (le zero e1)
                  (distinct one zero))))
      (a!2 (forall ((n Node) (e Epoch))
             (and (= (init.held n) (= n first))
                  (=> (distinct n first) (= (init.ep n) zero))
                  (= (init.ep first) one)
                  (= (init.transfer e n) false)
                  (= (init.locked e n) false))))
      (a!3 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (init.ep n1)) (not (le e2 (init.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1)))))
  (=> a!1 (=> a!2 a!3)))


In [13]:
def get_inductiveness_vc(M: DistLockModel, S1: DistLockState, action_fn, S2: DistLockState, inv_fn):
    vc = ( #Not(
        Implies(
            ForAll(*M.get_axioms()),
            Implies(
                And(
                    (inv_fn(M, S1)), # ForAll(*inv_fn(M, S1)),
                    ForAll(*action_fn(M, S1, S2)),
                ),
                (inv_fn(M, S2)) # ForAll(*inv_fn(M, S2))
            )
        )
    )
    return vc

def get_inductiveness_vc4unsat(M: DistLockModel, S1: DistLockState, action_fn, S2: DistLockState, inv_fn):
    vc = And( #Not(
        ForAll(*M.get_axioms()),
        (inv_fn(M, S1)), # ForAll(*inv_fn(M, S1)),
        ForAll(*action_fn(M, S1, S2)),
        Not(inv_fn(M, S2)) # ForAll(*inv_fn(M, S2))
    )
    return vc

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
inv_fn = lambda M,S: ForAll(*get_inv1(M, S))
vc = get_inductiveness_vc4unsat(M, S1, get_accept_action, S2, inv_fn)
print(vc.sexpr()) # only for testing

(let ((a!1 (forall ((e1 Epoch) (e2 Epoch) (e3 Epoch))
             (and (le e1 e1)
                  (=> (and (le e1 e2) (le e2 e3)) (le e1 e3))
                  (=> (and (le e1 e2) (le e2 e1)) (= e1 e2))
                  (or (le e1 e2) (le e2 e1))
                  (le zero e1)
                  (distinct one zero))))
      (a!2 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (pre.ep n1)) (not (le e2 (pre.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1))))
      (a!3 (forall ((n Node) (e Epoch))
             (let ((a!1 (and (pre.transfer e n) (not (le e (pre.ep n))))))
               (=> a!1 (and (post.held n) (= (post.ep n) e) (post.locked e n))))))
      (a!4 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (post.ep n1)) (not (le e2 (post.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1)))))
  (and a!1 a!2 a!3 (not a!4)))


In [14]:
def get_safety_vc(M: DistLockModel, S: DistLockState, inv_fn, safety_fn):
    vc = ( # Not(
        Implies(
            ForAll(*M.get_axioms()),
            Implies(
                (inv_fn(M, S)), #ForAll(*inv_fn(M, S)),
                ForAll(*safety_fn(M, S))
            )
        )
    )
    return vc

M = DistLockModel()
S1 = M.get_state('pre')
inv_fn = lambda M,S: ForAll(*get_inv1(M, S))
vc = get_safety_vc(M, S1, inv_fn, get_safety_inv)
print(vc.sexpr()) # only for testing

(let ((a!1 (forall ((e1 Epoch) (e2 Epoch) (e3 Epoch))
             (and (le e1 e1)
                  (=> (and (le e1 e2) (le e2 e3)) (le e1 e3))
                  (=> (and (le e1 e2) (le e2 e1)) (= e1 e2))
                  (or (le e1 e2) (le e2 e1))
                  (le zero e1)
                  (distinct one zero))))
      (a!2 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (pre.ep n1)) (not (le e2 (pre.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1))))
      (a!3 (forall ((e1 Epoch) (n1 Node) (n2 Node))
             (=> (and (pre.locked e1 n1) (pre.locked e1 n2)) (= n1 n2)))))
  (=> a!1 (=> a!2 a!3)))


In [15]:
# Inductiveness check using instantiatiojn
# ax & inv_pre & trans & \not inv_post
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv_post = And([inv(M,S2)[1] for inv in all_invars])
# print(inv_post)
inst_terms = extract_terms(inv_post)
# print(inst_terms)
# print(M.get_axioms())
s1 = Solver()
s1.add(M.get_axioms()[1])
print(s1)
print(s1.check())
ax_instantiation = instantiate(M.get_axioms(), inst_terms)
print(len(inst_terms[Epoch]))
# print(ax_instantiation)
# inv_pre_instantiation = And([instantiate(inv(M,S1), inst_terms) for inv in all_invars[:1]])
# trans_instantiation = instantiate(get_accept_action(M,S1,S2), inst_terms)
sol = Solver()
# inv_post
for inst in ax_instantiation:
    sol.add(inst)
    if sol.check() == sat:
        break
#print(sol)
print(sol.check())
# sol
# sol.add(ax_instantiation)
# sol
#sol.add(simplify(And(ax_instantiation, inv_pre_instantiation, trans_instantiation, Not(inv_post))))
# print('HereIam')
#print(sol.sexpr())
#sol
#sol.check()

[And(le(e1, e1),
     Implies(And(le(e1, e2), le(e2, e3)), le(e1, e3)),
     Implies(And(le(e1, e2), le(e2, e1)), e1 == e2),
     Or(le(e1, e2), le(e2, e1)),
     le(zero, e1),
     one != zero)]
sat
instantiating [109, 109, 109]
109
sat


In [16]:
s = Solver()
new = DeclareSort('new')
x = Const('x',new)
f = Function('f', new, IntSort())
s.add(And(f(x)==5))
s.check()
s.model()

[x = new!val!0, f = [else -> 5]]

## End to end examples

### Example 1

In [70]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M,S: And(ForAll(*all_invars[0](M,S)), ForAll(*all_invars[1](M,S)))

solver = Solver()
# solver.add()
cond = Not(And(
    #get_init_vc(M, inv),
    get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
))
#cond
solver.add(cond)
solver.check()

sat

The above tells that `get_inv1()` invariant satisfies the initial condition and is also inductive.

In [71]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M,S: ForAll(*get_inv1(M,S))

solver = Solver()
# solver.add()
solver.add(Not(And(
    get_init_vc(M, inv),
    get_inductiveness_vc(M, S1, get_accept_action, S2, inv),
    get_safety_vc(M, S1, inv, get_safety_inv)
)))
solver.check()

sat

However, it does not imply the safety condition.

### Example 2

In [73]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M,S: ForAll(*get_safety_inv(M,S)) # notice this <-- it's get_safety_inv and not get_inv1.

solver = Solver()
# solver.add()
solver.add(Not(And(
    get_init_vc(M, inv),
    # get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
)))
solver.check()

unsat

The safety invariant satisfies the initial condition.

In [74]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M,S: ForAll(*get_safety_inv(M,S)) # notice this <-- it's get_safety_inv and not get_inv1.

solver = Solver()
# solver.add()
solver.add(Not(And(
    get_init_vc(M, inv),
    get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
)))
solver.check()

sat

However, the safety invariant isn't inductive.

### Example 3

In [75]:
M = DistLockModel()
S1 = M.get_state('pre')

inv_fn = lambda M,S: ForAll(*get_inv1(M, S))

solver = Solver()
solver.add(Not(And(
    get_init_vc(M, inv_fn),
    get_safety_vc(M, S1, inv_fn, get_safety_inv))
))
solver.check()

sat

This shows that `get_inv1` doesn't imply safety condition.

The conditions given to the solver are shown below:

In [76]:
print(solver.sexpr())

(declare-sort Epoch 0)
(declare-sort Node 0)
(declare-fun pre.locked (Epoch Node) Bool)
(declare-fun le (Epoch Epoch) Bool)
(declare-fun pre.ep (Node) Epoch)
(declare-fun zero () Epoch)
(declare-fun one () Epoch)
(declare-fun init.ep (Node) Epoch)
(declare-fun init.locked (Epoch Node) Bool)
(declare-fun init.transfer (Epoch Node) Bool)
(declare-fun first () Node)
(declare-fun init.held (Node) Bool)
(assert (let ((a!1 (forall ((e1 Epoch) (e2 Epoch) (e3 Epoch))
             (and (le e1 e1)
                  (=> (and (le e1 e2) (le e2 e3)) (le e1 e3))
                  (=> (and (le e1 e2) (le e2 e1)) (= e1 e2))
                  (or (le e1 e2) (le e2 e1))
                  (le zero e1)
                  (distinct one zero))))
      (a!2 (forall ((n Node) (e Epoch))
             (and (= (init.held n) (= n first))
                  (=> (distinct n first) (= (init.ep n) zero))
                  (= (init.ep first) one)
                  (= (init.transfer e n) false)
                  (= (init

The counter example given by Z3 is shown below

In [77]:
m = solver.model()
print(m.sexpr())

;; universe for Epoch:
;;   Epoch!val!3 Epoch!val!2 Epoch!val!1 Epoch!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Epoch!val!3 () Epoch)
(declare-fun Epoch!val!2 () Epoch)
(declare-fun Epoch!val!1 () Epoch)
(declare-fun Epoch!val!0 () Epoch)
;; cardinality constraint:
(forall ((x Epoch))
        (or (= x Epoch!val!3)
            (= x Epoch!val!2)
            (= x Epoch!val!1)
            (= x Epoch!val!0)))
;; -----------
;; universe for Node:
;;   Node!val!0 Node!val!1 
;; -----------
;; definitions for universe elements:
(declare-fun Node!val!0 () Node)
(declare-fun Node!val!1 () Node)
;; cardinality constraint:
(forall ((x Node)) (or (= x Node!val!0) (= x Node!val!1)))
;; -----------
(define-fun one () Epoch
  Epoch!val!2)
(define-fun zero () Epoch
  Epoch!val!3)
(define-fun pre.locked ((x!0 Epoch) (x!1 Node)) Bool
  true)
(define-fun le ((x!0 Epoch) (x!1 Epoch)) Bool
  (or (and (= x!0 Epoch!val!3) (= x!1 Epoch!val!0) (not (= x!1 Epoch!val!3)))
      (and

Another way to access the counter example model is as shown below:

In [78]:
interp = M.get_interp(m)
set_param(max_lines=1000)
print(interp)

{'one': Epoch!val!2, 'zero': Epoch!val!3, 'pre.locked': [else -> True], 'le': [else ->
 Or(And(Var(0) == Epoch!val!3,
        Var(1) == Epoch!val!0,
        Not(Var(1) == Epoch!val!3)),
    And(Var(0) == Epoch!val!2,
        Not(Var(0) == Epoch!val!0),
        Not(Var(0) == Epoch!val!3),
        Not(Var(1) == Epoch!val!2),
        Not(Var(1) == Epoch!val!0),
        Not(Var(1) == Epoch!val!3)),
    And(Var(0) == Epoch!val!2,
        Not(Var(0) == Epoch!val!0),
        Not(Var(0) == Epoch!val!3),
        Var(1) == Epoch!val!2,
        Not(Var(1) == Epoch!val!0),
        Not(Var(1) == Epoch!val!3)),
    And(Not(Var(0) == Epoch!val!2),
        Not(Var(0) == Epoch!val!0),
        Not(Var(0) == Epoch!val!3),
        Not(Var(1) == Epoch!val!2),
        Not(Var(1) == Epoch!val!0),
        Not(Var(1) == Epoch!val!3)),
    And(Var(0) == Epoch!val!3,
        Var(1) == Epoch!val!2,
        Not(Var(1) == Epoch!val!0),
        Not(Var(1) == Epoch!val!3)),
    And(Var(0) == Epoch!val!3,
        Not(

## Full verification

This code below first checks if all the invariants produced by DistAI:
- Satisfy the initial state
- Imply Safety
- Are inductive

In [79]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[ForAll(*inv(M, S)) for inv in all_invars])

solver = Solver()
solver.add(Not(And(
    get_init_vc(M, inv),
    # get_safety_vc(M, S1, inv, get_safety_inv)
    # get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
)))
solver.check()

unsat

Yay, invariants satisfy the initial conditions

In [80]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[ForAll(*inv(M, S)) for inv in all_invars])

solver = Solver()
solver.add(Not(And(
    # get_init_vc(M, inv),
    get_safety_vc(M, S1, inv, get_safety_inv)
    # get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
)))
solver.check()

unsat

Yay, invariants imply the safety condition

In [84]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[ForAll(*inv(M, S)) for inv in all_invars])

solver = Solver()
cond1 = Not(And(
    # get_init_vc(M, inv),
    # get_safety_vc(M, S1, inv, get_safety_inv)
    get_inductiveness_vc(M, S1, get_accept_action, S2, inv)
))
cond1 = z3.simplify(cond1)

solver.add(cond1)
solver.check()

unknown

In [121]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[ForAll(*inv(M, S)) for inv in all_invars])

solver = Solver()
cond1 = get_inductiveness_vc4unsat(M, S1, get_accept_action, S2, inv)
cond1 = z3.simplify(cond1)

solver.add(cond1)
solver.check()

unknown

In [127]:
solver = Solver()
solver.add(Not(inv(M, S2)))
solver.check()

sat

And....Z3 chokes on the inductive invariants (wrt the accept action)

In [85]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv = lambda M, S: And(*[ForAll(*inv(M, S)) for inv in all_invars])

solver = Solver()
solver.add(Not(And(
    # get_init_vc(M, inv),
    # get_safety_vc(M, S1, inv, get_safety_inv)
    get_inductiveness_vc(M, S1, get_grant_action, S2, inv)
)))
solver.check()

unknown

It also chokes on the inductive invariants (wrt the grant action)

In [ ]:
def get_inductiveness_vc2(M: DistLockModel, S1: DistLockState, action_fn, S2: DistLockState, inv_fn):
    vars = []
    
    axiom_vars, axioms = M.get_axioms()
    inv_vars, invs = inv_fn(M, S1)
    action_vars, actions = action_fn(M, S1, S2)
    
    vc = ( #Not(
        Implies(
            ForAll(*M.get_axioms()),
            Implies(
                And(
                    (inv_fn(M, S1)), # ForAll(*inv_fn(M, S1)),
                    ForAll(*action_fn(M, S1, S2)),
                ),
                (inv_fn(M, S2)) # ForAll(*inv_fn(M, S2))
            )
        )
    )
    return vc

In [110]:
[instantiate(inv(M,S1), inst_terms) for inv in all_invars[:1]]

instantiating [80, 109, 109]


In [128]:
# Inductiveness check using instantiatiojn
# ax & inv_pre & trans & \not inv_post
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

inv_post = And([inv(M,S2)[1] for inv in all_invars])
inst_terms = extract_terms(inv_post)

ax_instantiation = instantiate(M.get_axioms(), inst_terms)
pre_instantiation = [instantiate(inv(M,S1), inst_terms) for inv in all_invars]
trans_instantiation = instantiate(get_accept_action(M, S1, S2), inst_terms)

sol = Solver()
sol.add(Not(inv_post))
print(sol.check())

for inst1, pre_insts, inst3 in zip(ax_instantiation, pre_instantiation, trans_instantiation):
    sol.add(inst1)
    sol.add(inst3)
    if sol.check() == unsat:
        break
    for inst2 in pre_insts:
        sol.add(inst2)
        if sol.check() == unsat:
            break
# inv_post
# for inst in ax_instantiation:
#     sol.add(inst)
#     if sol.check() == sat:
#         break
# #print(sol)
# print(sol.check())
# sol
# sol.add(ax_instantiation)
# sol
#sol.add(simplify(And(ax_instantiation, inv_pre_instantiation, trans_instantiation, Not(inv_post))))
# print('HereIam')
#print(sol.sexpr())
#sol
#sol.check()

instantiating [109, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 80, 109, 109]
instantiating [80, 109]
instantiating [80, 109]
instantiating [80, 109]
instantiating [80, 109]
instantiating [80, 109]
instantiating [80, 80]
instantiating [80, 80, 109]
instantiating [80, 80, 109]
instantiating [80, 80]
instantiating [80, 80]
instantiating [80, 80, 109]
instantiating [80, 80]
instantiating [80, 80, 109]
instantiating [80, 80]
instantiating [80, 8

Even the above form of lazy term instantiations flops.